### 5.3 Programming Task: Digit recognition using CNNs

In [1]:
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms
from torchinfo import summary
import torchvision

%matplotlib inline

i. Complete the code for the ConvNet class given below using the network description from supplement pdf.

In [2]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()    
        self.conv = nn.Conv2d(in_channels=1,out_channels=20, kernel_size=(5,5),stride=(1,1),padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.fc1  = nn. Linear(20*13*13,100)
        self.fc2  = nn. Linear(100,10)

        
    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x,dim=1) 




Show the net.

In [3]:
net = ConvNet()
print(net)

ConvNet(
  (conv): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3380, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


ii. Train the CNN and observe the difference in performance in comparison to the feed-forward
network from the task 5.2.

In [4]:
batch_size=200
learning_rate=0.01
epochs=10

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/files/', train=True, download=True,
                   transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/files/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=batch_size, shuffle=True)

In [6]:
# Set the loss function and the optimization criteria

optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

criterion = nn.NLLLoss()

In [7]:
# Run the main training loop

for epoch in range(epochs):
    total_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # reshape data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
        
        optimizer.zero_grad()
        
        net_out = net(data)
        
        loss = criterion(net_out, target)
        total_loss+=loss.data
        
        loss.backward()
        optimizer.step()

    print('Train Epoch: {} Avg. Training Loss: {:.6f}\n'.format(
                epoch+1,
                total_loss/len(train_loader.dataset)))

Train Epoch: 1 Avg. Training Loss: 0.001657

Train Epoch: 2 Avg. Training Loss: 0.000464

Train Epoch: 3 Avg. Training Loss: 0.000319

Train Epoch: 4 Avg. Training Loss: 0.000242

Train Epoch: 5 Avg. Training Loss: 0.000210

Train Epoch: 6 Avg. Training Loss: 0.000168

Train Epoch: 7 Avg. Training Loss: 0.000147

Train Epoch: 8 Avg. Training Loss: 0.000120

Train Epoch: 9 Avg. Training Loss: 0.000107

Train Epoch: 10 Avg. Training Loss: 0.000097



In [8]:
# Run the testing loop

test_loss = 0.0
correct = 0.0

for data, target in test_loader:
    
    
    net_out = net(data)
    
    # sum up batch loss
    test_loss += criterion(net_out, target).data
    
    pred = net_out.data.max(dim=1)[1]  # get the index of the max log probability. Here [1] returns the argmax. 
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {:.0f}/{} ({:.2%})\n'.format(
    test_loss, correct, len(test_loader.dataset),correct / len(test_loader.dataset)))


Test set: Average loss: 0.0002, Accuracy: 9880/10000 (98.80%)



iii. Calculate the number of learnable parameters and the output shape in each layer. Verify your
answers with model summary. (Refer last cell of the tutorial notebook)

In [26]:
summary(net, input_size=(200,1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
ConvNet                                  [200, 10]                 --
├─Conv2d: 1-1                            [200, 20, 26, 26]         520
├─MaxPool2d: 1-2                         [200, 20, 13, 13]         --
├─Linear: 1-3                            [200, 100]                338,100
├─Linear: 1-4                            [200, 10]                 1,010
Total params: 339,630
Trainable params: 339,630
Non-trainable params: 0
Total mult-adds (M): 138.13
Input size (MB): 0.63
Forward/backward pass size (MB): 21.81
Params size (MB): 1.36
Estimated Total Size (MB): 23.79